In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [55]:
class BlurDetectionModel(nn.Module):
    def __init__(self):
        super(BlurDetectionModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.dropout = nn.Dropout(0.5)
        self.fc1 = nn.Linear(128 * 28 * 28, 128)
        self.fc2 = nn.Linear(128, 1)
        
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1, 128 * 28 * 28)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)  # Apply dropout to reduce overfitting
        x = torch.sigmoid(self.fc2(x))
        return x

In [56]:
model = BlurDetectionModel()

In [57]:
# Define the loss function and optimizer
criterion = nn.BCELoss()  # Binary Cross-Entropy Loss for binary classification
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)  # Adam optimizer

In [58]:
from torchvision import transforms
from PIL import Image
from torch.utils.data import DataLoader, Dataset

In [59]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),  # Data augmentation
    transforms.RandomRotation(10),      # Data augmentation
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [49]:
import os
%pwd

'/mnt/c/Users/sv/source/Enhancement/Image-Enchancer'

In [32]:
os.listdir()

['Image-Enchancer']

In [39]:
os.chdir("./Image-Enchancer")

In [60]:
image = Image.open('.//Experiments//Non_Blurry_folder//image_34.jpg')
image = transform(image).unsqueeze(0)

In [61]:
model.eval()
with torch.no_grad():
    output = model(image)

In [62]:
prediction = torch.round(output).item()  # 0 or 1 for blur detection
if prediction == 1:
    print("The image is blurry.")
else:
    print("The image is not blurry.")

The image is blurry.


In [ ]:
rspnet